In [1]:
import os
from PIL import Image
import glob
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(224,224)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")


# 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path1 = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
image_dir_path2 = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
image_dir_path3 = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
resize_images(image_dir_path1)
resize_images(image_dir_path2)
resize_images(image_dir_path3)
image_dir_path_test1 = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/scissor"
image_dir_path_test2 = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/rock"
image_dir_path_test3 = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/paper"
resize_images(image_dir_path_test1)
resize_images(image_dir_path_test2)
resize_images(image_dir_path_test3)

def load_data(img_path, number_of_data):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=224
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path, 502)
image_dir_path_test = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
(x_test, y_test)=load_data(image_dir_path_test, 300)
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))


179  images to be resized.
179  images resized.
182  images to be resized.
182  images resized.
141  images to be resized.
141  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
학습데이터(x_train)의 이미지 개수는 502 입니다.
학습데이터(x_train)의 이미지 개수는 300 입니다.
x_train shape: (502, 224, 224, 3)
y_train shape: (502,)


In [2]:
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224,224,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(256, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(256, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

print("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))
print("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))

x_train_reshaped=x_train_norm.reshape( -1, 224, 224, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
x_test_reshaped=x_test_norm.reshape( -1, 224, 224, 3)

print("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))
print("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])


Before Reshape - x_train_norm shape: (502, 224, 224, 3)
Before Reshape - x_test_norm shape: (300, 224, 224, 3)
After Reshape - x_train_reshaped shape: (502, 224, 224, 3)
After Reshape - x_test_reshaped shape: (300, 224, 224, 3)


In [3]:
# 모델 훈련
model.fit(x_train_reshaped, y_train, epochs=15)

# 모델 시험
test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))


Epoch 1/15
16/16 [==============================] - 4s 55ms/step - loss: 1.5256 - accuracy: 0.3446
Epoch 2/15
16/16 [==============================] - 0s 26ms/step - loss: 1.1272 - accuracy: 0.3665
Epoch 3/15
16/16 [==============================] - 0s 26ms/step - loss: 1.1314 - accuracy: 0.3327
Epoch 4/15
16/16 [==============================] - 0s 26ms/step - loss: 1.0998 - accuracy: 0.3566
Epoch 5/15
16/16 [==============================] - 0s 26ms/step - loss: 1.0827 - accuracy: 0.4502
Epoch 6/15
16/16 [==============================] - 0s 26ms/step - loss: 1.0367 - accuracy: 0.4502
Epoch 7/15
16/16 [==============================] - 0s 25ms/step - loss: 0.9036 - accuracy: 0.5359
Epoch 8/15
16/16 [==============================] - 0s 25ms/step - loss: 0.8567 - accuracy: 0.5498
Epoch 9/15
16/16 [==============================] - 0s 25ms/step - loss: 0.6936 - accuracy: 0.6295
Epoch 10/15
16/16 [==============================] - 0s 26ms/step - loss: 0.7262 - accuracy: 0.5916
Epoch 11/